In [1]:
import numpy as np
import torch
import datetime
import os
import base64
import pandas as pd
from collections import Counter
from tqdm import tqdm
import sqlite3
from utils.pipeline import get_files
from utils.tools import seconds_to_time
from utils.data_analysis import get_overlap_undefined,get_direction_info
# Set the maximum number of rows and columns to display
pd.set_option('display.max_rows', 1000)  # Adjust the number as needed
pd.set_option('display.max_columns', 1000)  # Adjust the number as needed

files = get_files('/home/diego/Documents/yolov7-tracker/runs/detect/2024_04_17_conce_bytetrack')
db = files['db']
FRAME_NUMBER = 15
conn = sqlite3.connect(db)
bbox = pd.read_sql('SELECT * FROM bbox_raw', conn)
bbox['direction'] = bbox.apply(lambda row: ('undefined' if row['img_name'].split('_')[3] == 'None' else  row['img_name'].split('_')[3]) if row['img_name'] is not None else None, axis=1)
bbox['time_sec'] = bbox.apply(lambda row: int(row['frame_number']) // FRAME_NUMBER, axis=1)
bbox['time_video'] = pd.to_datetime(bbox['time_sec'], unit='s').dt.time


In [2]:
get_direction_info(bbox)

{'undefined': 1466, 'out': 590, 'in': 565, 'cross': 64}

In [3]:
overlap_results = get_overlap_undefined(bbox, 0,['undefined'])
overlap_results.to_csv('overlap_results_undefined_2.csv', index=False)

overlap_results = get_overlap_undefined(bbox, 0,['In', 'Out'])
overlap_results.to_csv('overlap_results_in_out_2.csv', index=False)

In [ ]:
overlap_results['overlap_type'].value_counts()

In [ ]:
analysis.to_csv('analysis.csv', index=False)

In [ ]:
intervals = group_by_interval(filtered_df)
total = count_ids_in_intervals(intervals)
#print(total)

with open('intervals.json', 'w') as f:
	f.write(str(intervals))
 
with open('total.json', 'w') as f:
	f.write(str(total))

In [ ]:
intervals[35750]

In [ ]:
filtered_df.to_csv('bbox_2.csv', index=False)

### Ex

In [ ]:
def group_by_interval(df):
    temp_df = df.dropna(subset=['img_name']).copy()
    group = temp_df.groupby(['time_interval', 'id']).agg({'direction': 'first'}).reset_index()
    
    interval_dict = {}
    for _, row in group.iterrows():
        interval = row['time_interval']
        if interval not in interval_dict:
            interval_dict[interval] = {}
        interval_dict[interval][row['id']] = {'direction': row['direction']}
    return interval_dict

# filter_interval_range_with_only_none
def get_intervals_with_none(interval_dict):
    none_intervals = {}
    only_ids = set()
    for interval, ids in interval_dict.items():
        all_none = True
        for id_info in ids.values():
            if id_info['direction'] != 'undefined':
                all_none = False
                break
        if all_none:
            none_intervals[interval] = ids.copy()  # Create a copy of ids if ids itself is mutable
            only_ids.update(ids.keys())
    return none_intervals, only_ids

def remove_ids_from_df(df, ids_to_remove):
    # Use the `~` operator to select rows where 'id' is not in 'ids_to_remove'
    filtered_df = df[~df['id'].isin(ids_to_remove)].copy()  # Explicitly copying is optional here since filtering creates a new df
    return filtered_df

def count_ids_in_intervals(interval_dict):
    interval_counts = {interval: len(ids) for interval, ids in interval_dict.items()}
    sorted_interval_counts = dict(sorted(interval_counts.items(), key=lambda item: item[1], reverse=True))
    return sorted_interval_counts

info_before = get_direction_info(bbox)
print(f"Info before: {info_before}")
df = add_column_time_interval(bbox, interval_sec=5, frame_rate=15)
result = group_by_interval(df)
intervals_keys,ids = get_intervals_with_none(result)
filtered_df = remove_ids_from_df(df, ids)
info_after = get_direction_info(filtered_df)
print(f"Info after: {info_after}")